<a href="https://colab.research.google.com/github/g-roger/natural-language-process/blob/main/Resolucao_Aula_4_6_IA_PLN_Exercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Exercício 1**

###Que tal fazer isso com seu melhor modelo classificador de produtos?

- Crie um pipeline produtivo do seu melhor modelo até aqui.

Carregamento dos dados e divisão dos dados entre amostras de Treino e Teste

In [ ]:
# Pacotes utilizados
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga dos dados
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

# Limpeza da base de dados
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

# Divisão das amostras em treino e teste
df_treino, df_teste = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

###**Parte 1: Pipeline de treinamento**

Instalação das dependências

In [ ]:
# Instalação do pacote SpaCy (se necessário)
!pip install spacy
!python -m spacy download pt

Preparação dos dados de **TREINO**

In [ ]:
# Pacotes utilizados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy
import nltk

# Dependências do NLTK e do Spacy
nltk.download('stopwords')
nlp = spacy.load('pt')

# Função de lematização para os verbos do documento
def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

# Aplica a lematização no dataframe criando uma nova coluna
df_treino['text_lemma_verbs'] = df_treino.texto.apply(lemmatizer_verbs)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Vetorização dos dados de **TREINO** e Treinamento do modelo

In [ ]:
# Vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento com verbos lematizado, sem stopwords do SpaCy e NLTK combinadas e modelo de classificação DecisionTreeClassifier

# Carrega as stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# vetorização por contagem de termos no documento com os verbos lematizado
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama e bigrama, remoção das stopwords NLTK e Spacy
vect.fit(df_treino.text_lemma_verbs)
text_vect_treino = vect.transform(df_treino.text_lemma_verbs)

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect_treino, df_treino.categoria) # text_vect_treino: vetor com os dados de treino lematizado | df_treino.categoria: classificação real dos dados para o treinamento do modelo

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

Salvando o vetor e o modelo treinado para reutilizar no pipeline produtivo

In [ ]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Nov  1 13:35 .
drwxr-xr-x 1 root root 4096 Nov 10 12:39 ..
drwxr-xr-x 4 root root 4096 Nov  1 13:34 .config
drwxr-xr-x 1 root root 4096 Nov  1 13:35 sample_data


In [ ]:
import pickle

pickle.dump(tree, open('minha_arvore.pkl', 'wb'))
pickle.dump(vect, open('meu_vetorizador.pkl', 'wb'))

In [ ]:
!ls -la

total 7884
drwxr-xr-x 1 root root    4096 Nov 10 12:43 .
drwxr-xr-x 1 root root    4096 Nov 10 12:39 ..
drwxr-xr-x 4 root root    4096 Nov  1 13:34 .config
-rw-r--r-- 1 root root 8034789 Nov 10 12:43 meu_vetorizador.pkl
-rw-r--r-- 1 root root   17484 Nov 10 12:43 minha_arvore.pkl
drwxr-xr-x 1 root root    4096 Nov  1 13:35 sample_data


###**Parte 2: Pipeline simulando um cenário produtivo**

Preparação dos dados de **TESTE**, Vetorização e classificação com base no modelo treinado

In [ ]:
# Pacotes utilizados nesse pipeline
import spacy
import nltk
import pickle

# Função de lematização para os verbos do documento
def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

# Aplica a lematização no dataframe criando uma nova coluna
df_teste['text_lemma_verbs'] = df_teste.texto.apply(lemmatizer_verbs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Carrega o vetor treinado
meu_vetorizador = pickle.load(open('meu_vetorizador.pkl', 'rb'))

# vetorização no dataframe de teste
text_vect_teste = meu_vetorizador.transform(df_teste.text_lemma_verbs)

# Carrega o modelo treinado
minha_arvore = pickle.load(open('minha_arvore.pkl', 'rb'))

# Escoragem da classificação na amostra de teste
y_predicao = minha_arvore.predict(text_vect_teste)

# Escoragem no dataframe
df_teste['categoria_predita'] = y_predicao
df_teste.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,nome,descricao,categoria,texto,text_lemma_verbs,categoria_predita
2700,Estojo Duo Iluminador E Bronzer Belle Angel M...,DUO ILUMINADOR E BRONZER BELLE ANGEL - B025O D...,maquiagem,Estojo Duo Iluminador E Bronzer Belle Angel M...,Estojo Duo Iluminador E Bronzer Belle Angel ...,maquiagem
1297,Patrulha Canina Carrinhos De Fricção 6 Person...,Kit carrinhos da Patrulha canina Tamanho: Cerc...,brinquedo,Patrulha Canina Carrinhos De Fricção 6 Person...,Patrulha Canina Carrinhos De Fricção 6 Perso...,brinquedo
3162,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,#Nossos produtos são NOVOS e 100% ORIGINAIS# ...,game,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,Pokemon Ultra Moon Nintendo 3ds Midia Fisica...,game
2895,Kite Pincel 12 Unidade,esta alta qualidade Punho de madeira Escova Co...,maquiagem,Kite Pincel 12 Unidade esta alta qualidade ...,Kite Pincel 12 Unidade esta alta qualida...,maquiagem
2226,Maleta Grande P/ Maquiagem Profissional Rodin...,FOTOS REAIS DO PRODUTO - ENVIAMOS SUA MALETA C...,maquiagem,Maleta Grande P/ Maquiagem Profissional Rodin...,Maleta Grande P/ Maquiagem Profissional Rodi...,maquiagem


In [ ]:
# Mensuração do resultado pela acurácia
y_teste = df_teste.categoria
accuracy = accuracy_score(y_predicao, y_teste)

print(accuracy)

0.96


In [ ]:
# Exemplo escoragem com um documento
texto = 'Estojo Duo Iluminador E Bronzer'
text_lemma = lemmatizer_verbs(texto)
print(text_lemma)

text_vect_teste = meu_vetorizador.transform([text_lemma])
y_predicao = minha_arvore.predict(text_vect_teste)
print(y_predicao)

Estojo Duo Iluminador E Bronzer
['maquiagem']
